In [1]:
#cell to import libraries 
import pandas as pd 
import numpy as np
import re
import nltk
import  matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import wordnet as wn
from collections import Counter
from sklearn.metrics.pairwise import cosine_similarity
from nltk.stem.porter import PorterStemmer

import ast
from sklearn.model_selection import train_test_split


In [3]:
#importing dataset
credits = pd.read_csv('tmdb_5000_credits.csv')
movies = pd.read_csv('tmdb_5000_movies.csv')

In [49]:
credits.dtypes

movie_id     int64
title       object
cast        object
crew        object
dtype: object

In [15]:
movies.dtypes

budget                    int64
genres                   object
homepage                 object
id                        int64
keywords                 object
original_language        object
original_title           object
overview                 object
popularity              float64
production_companies     object
production_countries     object
release_date             object
revenue                   int64
runtime                 float64
spoken_languages         object
status                   object
tagline                  object
title                    object
vote_average            float64
vote_count                int64
dtype: object

In [11]:
credits.shape

(4803, 4)

In [17]:
movies.shape

(4803, 20)

In [7]:
#reviewing the dataframe imported\
credits.head()

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


We can see that the review text has unwanted characters like < , / , so on... 
Let's try to remove that. 
We are not removing any numbers because of the possibility that the reviews might contain rating like : 3/10 or 10/10, which might prove useful.

In [9]:
movies.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124


In [383]:
#checking for null values
credits.isna().sum()

movie_id    0
title       0
cast        0
crew        0
dtype: int64

In [19]:
#checking for null values
movies.isna().sum()

budget                     0
genres                     0
homepage                3091
id                         0
keywords                   0
original_language          0
original_title             0
overview                   3
popularity                 0
production_companies       0
production_countries       0
release_date               1
revenue                    0
runtime                    2
spoken_languages           0
status                     0
tagline                  844
title                      0
vote_average               0
vote_count                 0
dtype: int64

In [5]:
# Merge the movies and credits dataframes on the 'title' column
movies = movies.merge(credits,on='title')

In [25]:
movies.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,movie_id,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...",...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,285,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...",...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,206647,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...",...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,49026,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]",...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,49529,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [55]:
movies.columns

Index(['budget', 'genres', 'homepage', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count', 'movie_id', 'cast', 'crew'],
      dtype='object')

We can remove a few columns like budget, homepage, etc.

In [7]:
#Removing columns from dataset.
columns_to_remove = ['budget', 'vote_count','homepage', 'original_language', 'original_title', 'popularity',
                     'production_countries', 'release_date', 'revenue',
                     'runtime', 'spoken_languages','tagline','id','movie_id']
movies.drop(columns=columns_to_remove, inplace=True)

In [211]:
# Check for null values in each column
movies.isnull().sum()

genres                  0
keywords                0
overview                3
production_companies    0
status                  0
title                   0
vote_average            0
cast                    0
crew                    0
dtype: int64

Only 3 out of 4803 movies do not have an overview, which is fine. So I've decided to keep the null values without imputing or removing them. 

In [61]:
# Check for duplicate rows
movies.duplicated().sum()

0

In [27]:
movies['keywords'][0]

'[{"id": 1463, "name": "culture clash"}, {"id": 2964, "name": "future"}, {"id": 3386, "name": "space war"}, {"id": 3388, "name": "space colony"}, {"id": 3679, "name": "society"}, {"id": 3801, "name": "space travel"}, {"id": 9685, "name": "futuristic"}, {"id": 9840, "name": "romance"}, {"id": 9882, "name": "space"}, {"id": 9951, "name": "alien"}, {"id": 10148, "name": "tribe"}, {"id": 10158, "name": "alien planet"}, {"id": 10987, "name": "cgi"}, {"id": 11399, "name": "marine"}, {"id": 13065, "name": "soldier"}, {"id": 14643, "name": "battle"}, {"id": 14720, "name": "love affair"}, {"id": 165431, "name": "anti war"}, {"id": 193554, "name": "power relations"}, {"id": 206690, "name": "mind and soul"}, {"id": 209714, "name": "3d"}]'

In [29]:
movies['genres'][0]

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

In [ ]:
movies['cast'][0]

In [ ]:
movies['crew'][0]

In [47]:
movies['production_companies'][0]

'[{"name": "Ingenious Film Partners", "id": 289}, {"name": "Twentieth Century Fox Film Corporation", "id": 306}, {"name": "Dune Entertainment", "id": 444}, {"name": "Lightstorm Entertainment", "id": 574}]'

In [9]:
# Define a function to convert the genre/keyword JSON string into a list of genre/keyword names
def convertname(text):
    L = []
    for i in ast.literal_eval(text):
        L.append(i['name']) 
    return L

In [11]:
def converttennames(obj):
    L = []
    counter = 0 
    for i in ast.literal_eval(obj):
        if counter != 10:
            L.append(i['name'])
            counter +=1
        else:
            break
    return L

In [13]:
def convertdirector(obj):
    l = []
    for i in ast.literal_eval(obj):
        if i['job'] == 'Director':
            l.append(i['name'])
            break
    return l

In [15]:
movies['keywords'] = movies['keywords'].apply(convertname)

In [59]:
movies['keywords'][0]

['culture clash',
 'future',
 'space war',
 'space colony',
 'society',
 'space travel',
 'futuristic',
 'romance',
 'space',
 'alien',
 'tribe',
 'alien planet',
 'cgi',
 'marine',
 'soldier',
 'battle',
 'love affair',
 'anti war',
 'power relations',
 'mind and soul',
 '3d']

In [17]:
movies['genres'] = movies['genres'].apply(convertname)

In [61]:
movies['genres'][0]

['Action', 'Adventure', 'Fantasy', 'Science Fiction']

In [19]:
movies['production_companies'] = movies['production_companies'].apply(convertname)

In [137]:
movies['production_companies'][0]

['Ingenious Film Partners',
 'Twentieth Century Fox Film Corporation',
 'Dune Entertainment',
 'Lightstorm Entertainment']

In [21]:
movies['cast'] = movies['cast'].apply(converttennames)

In [83]:
movies['cast'][0]

['Sam Worthington',
 'Zoe Saldana',
 'Sigourney Weaver',
 'Stephen Lang',
 'Michelle Rodriguez',
 'Giovanni Ribisi',
 'Joel David Moore',
 'CCH Pounder',
 'Wes Studi',
 'Laz Alonso']

In [22]:
movies['crew'] = movies['crew'].apply(convertdirector)

In [77]:
movies['crew'][0]

['James Cameron']

In [227]:
movies

,genres,keywords,overview,production_companies,status,title,vote_average,cast,crew
0,"[Action, Adventure, Fantasy, Science Fiction]","['cultur clash', 'future', 'space war', 'space...","In the 22nd century, a paraplegic Marine is di...","[Ingenious Film Partners, Twentieth Century Fo...",Released,Avatar,7.2,"[Sam Worthington, Zoe Saldana, Sigourney Weave...",[James Cameron]
1,"[Adventure, Fantasy, Action]","['ocean', 'drug abuse', 'exot island', 'east i...","Captain Barbossa, long believed to be dead, ha...","[Walt Disney Pictures, Jerry Bruckheimer Films...",Released,Pirates of the Caribbean: At World's End,6.9,"[Johnny Depp, Orlando Bloom, Keira Knightley, ...",[Gore Verbinski]
2,"[Action, Adventure, Crime]","['spy', 'base on novel', 'secret agent', 'sequ...",A cryptic message from Bond’s past sends him o...,"[Columbia Pictures, Danjaq, B24]",Released,Spectre,6.3,"[Daniel Craig, Christoph Waltz, Léa Seydoux, R...",[Sam Mendes]
3,"[Action, Crime, Drama, Thriller]","['dc comics', 'crime fighter', 'terrorist', 's...",Following the death of District Attorney Harve...,"[Legendary Pictures, Warner Bros., DC Entertai...",Released,The Dark Knight Rises,7.6,"[Christian Bale, Michael Caine, Gary Oldman, A...",[Christopher Nolan]
4,"[Action, Adventure, Science Fiction]","['base on novel', 'mars', 'medallion', 'space ...","John Carter is a war-weary, former military ca...",[Walt Disney Pictures],Released,John Carter,6.1,"[Taylor Kitsch, Lynn Collins, Samantha Morton,...",[Andrew Stanton]
...,...,...,...,...,...,...,...,...,...
4804,"[Action, Crime, Thriller]","['unit states–mexico barrier', 'legs', 'arms',...",El Mariachi just wants to play his guitar and ...,[Columbia Pictures],Released,El Mariachi,6.6,"[Carlos Gallardo, Jaime de Hoyos, Peter Marqua...",[Robert Rodriguez]
4805,"[Comedy, Romance]",[],A newlywed couple's honeymoon is upended by th...,[],Released,Newlyweds,5.9,"[Edward Burns, Kerry Bishé, Marsha Dietlein, C...",[Edward Burns]
4806,"[Comedy, Drama, Romance, TV Movie]","['date', 'love at first sight', 'narration', '...","""Signed, Sealed, Delivered"" introduces a dedic...","[Front Street Pictures, Muse Entertainment Ent...",Released,"Signed, Sealed, Delivered",7.0,"[Eric Mabius, Kristin Booth, Crystal Lowe, Geo...",[Scott Smith]
4807,[],[],When ambitious New York attorney Sam is sent t...,[],Released,Shanghai Calling,5.7,"[Daniel Henney, Eliza Coupe, Bill Paxton, Alan...",[Daniel Hsia]


In [24]:
movies['tags'] = movies['genres'].astype(str) + movies['keywords'].astype(str) + movies['overview'].astype(str) + movies['cast'].astype(str) + movies['crew'].astype(str)

In [27]:
df = movies[['title','tags','vote_average']]

In [103]:
df.head()

,title,tags,vote_average
0,Avatar,"['Action', 'Adventure', 'Fantasy', 'Science Fi...",7.2
1,Pirates of the Caribbean: At World's End,"['Adventure', 'Fantasy', 'Action']['ocean', 'd...",6.9
2,Spectre,"['Action', 'Adventure', 'Crime']['spy', 'based...",6.3
3,The Dark Knight Rises,"['Action', 'Crime', 'Drama', 'Thriller']['dc c...",7.6
4,John Carter,"['Action', 'Adventure', 'Science Fiction']['ba...",6.1


In [29]:
cv = CountVectorizer(max_features=5000,stop_words='english')
ps = PorterStemmer()

In [31]:
vectors = cv.fit_transform(df['tags']).toarray()

In [39]:
vectors.shape

(4809, 5000)

Cosine Similarity 

In [33]:
similarity = cosine_similarity(vectors)

In [261]:
similarity[0]

array([1.        , 0.0579771 , 0.04272655, ..., 0.03008284, 0.05081827,
       0.        ])

In [35]:
sorted(list(enumerate(sorted(similarity[0]))), reverse=True,key = lambda x:x[1])[1:6]

[(4807, 0.42183074386605374),
 (4806, 0.3957898639029131),
 (4805, 0.3783020418228755),
 (4804, 0.3673278155844316),
 (4803, 0.3510107635810227)]

In [37]:
def movie_recommend(movie):
    # Filter to find rows the genre
    movie_index = df[df['title'].astype(str).str.contains(str(movie), case=False, na=False)].index
    
    # If no genre is found, return a message
    if len(movie_index) == 0:
        print("No movies found for this genre.")
        return
    
    # Get the first index matching the genre
    first_movie_index = movie_index[0]
    
    # Calculate distances for these indexes
    distances = similarity[first_movie_index]
    
    # Sort the distances and get the top 5 similar movies
    movie_list = sorted(list(enumerate(distances)), reverse=True, key=lambda x: x[1])[1:6]
    
    # Print the recommended movie titles
    for i in movie_list:
        print(df.iloc[i[0]].title)

    return

In [39]:
movie_recommend('Avatar')

Aliens
Moonraker
Alien
Alien³
Lifeforce


In [41]:
movie_recommend('Batman')

Man of Steel
Superman
Batman Begins
The Dark Knight
Batman Returns


In [43]:
def genre_recommend(genre):
    # Filter to find rows the genre
    genre_index = df[df['tags'].astype(str).str.contains(str(genre), case=False, na=False)].index
    
    # If no genre is found, return a message
    if len(genre_index) == 0:
        print("No movies found for this genre.")
        return
    
    # Get the first index matching the genre
    first_genre_index = genre_index[0]
    
    # Calculate distances for these indexes
    distances = similarity[first_genre_index]
    
    # Sort the distances and get the top 5 similar movies
    genre_list = sorted(list(enumerate(distances)), reverse=True, key=lambda x: x[1])[1:6]
    
    # Print the recommended movie titles
    for i in genre_list:
        print(df.iloc[i[0]].title)

    return

In [45]:
genre_recommend('action')

Aliens
Moonraker
Alien
Alien³
Lifeforce


In [49]:
genre_recommend('sci-fi')

Terminator Genisys
Logan's Run
Megiddo: The Omega Code 2
The Terminator
Terminator 2: Judgment Day


In [51]:
genre_recommend('romance')

Aliens
Moonraker
Alien
Alien³
Lifeforce


In [61]:
def actorsimilarmovies_recommend(actor):
    # Filter to find rows where the actor's name is in the tags
    actor_index = df[df['tags'].astype(str).str.contains(str(actor), case=False, na=False)].index
    
    # If no movies are found with this actor, return a message
    if len(actor_index) == 0:
        print("No movies found for this actor.")
        return
    
    # Accumulate the distances from all found actor indexes
    accumulated_distances = None
    for idx in actor_index:
        distances = similarity[idx]
        if accumulated_distances is None:
            accumulated_distances = distances
        else:
            accumulated_distances += distances
    
    # Sort the distances and get the top 5 similar movies
    actor_list = sorted(list(enumerate(accumulated_distances)), reverse=True, key=lambda x: x[1])[1:6]
    
    for i in actor_list:
        print(df.iloc[i[0]].title)

    return

In [63]:
actorsimilarmovies_recommend('Zoe Saldana')

Star Trek Into Darkness
Star Trek
Guardians of the Galaxy
Colombiana
Avatar


In [71]:
actorsimilarmovies_recommend('Taylor Swift')

Hannah Montana: The Movie
Allegiant
Hoot
If I Stay
Blood and Chocolate


In [73]:
actorsimilarmovies_recommend('Ryan Reynolds')

The Croods
Adventureland
Smokin' Aces
X-Men Origins: Wolverine
The Proposal


In [75]:
# Import the pickle module for saving the model
import pickle
# Save the modified dataframe to a pickle file
pickle.dump(df,open('movie_recommender.pkl','wb'))

# Save the similarity matrix to a pickle file
pickle.dump(similarity,open('similar_movies.pkl','wb'))